# Image normalization and scattering correction

## Load the needed modules

We first need to load some modules to work with this notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
import amglib.readers as rd
import amglib.bb as bb
from skimage.measure import label, regionprops
import pandas as pd
import skimage.morphology as morph
import skimage.filters as filt
from skimage.feature import match_template
from skimage.draw import disk

The reload is mainly for the development of the support modules used in this notebook.

In [ ]:
import importlib
importlib.reload(rd)
importlib.reload(bb)

## Load some test images
In this notebook we will work with the following images.

In [ ]:
img  = rd.readImage('../../../TestData/2D/fits/BB/sample_0001.fits') 
dc   = rd.readImage('../../../TestData/2D/fits/BB/dc_00001.fits')
ob   = rd.readImage('../../../TestData/2D/fits/BB/ob_00001.fits')
bbs  = rd.readImage('../../../TestData/2D/fits/BB/bbsample_00001.fits')
bbob = rd.readImage('../../../TestData/2D/fits/BB/bbob_00001.fits')

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(15,5))

vmin = 300
vmax = 10000
cmap = 'gray'
ax[0].imshow(dc,   vmin=200,vmax=400,   cmap=cmap); ax[0].set_title('DC')
ax[1].imshow(ob,   vmin=vmin,vmax=vmax, cmap=cmap); ax[1].set_title('OB')
ax[2].imshow(bbs,  vmin=vmin,vmax=vmax, cmap=cmap); ax[2].set_title('BB Sample')
ax[3].imshow(bbob, vmin=vmin,vmax=vmax, cmap=cmap); ax[3].set_title('BB OB')
ax[4].imshow(img,  vmin=vmin,vmax=vmax, cmap=cmap); ax[4].set_title('Sample');

# Normalization
The basic normalization of images is done using the folowing expression:

$$T=\frac{Img-DC}{OB-DC}$$

It is expected that the image intensity is equal to one in areas that are not covered by any material.

This does not hold when the source intensity is varying between the images. The source is mostly flucuating to some degree, therefore it is good practice to include a dose correction factor 

$$T_{dose}=\frac{D(OB,roi)}{D(Img,roi)}\frac{Img-DC}{OB-DC}$$

where the dose is computed as the average intensity of a region which is not covered by any material. The average is senitive to outliers. The median of the region can be used to make the dose calculation more robust against outliers, or like here the average of medians:

$$D(f,roi)=mean[median_{rows}[f(roi)]]$$


In [ ]:
nimg = bb.normalize(img,ob,dc, logarithm=False)
N=25
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

fig, ax = plt.subplots(1,3,figsize=(15,4))
ax[0].imshow(nimg,vmin=0,vmax=1.25,cmap='gray')
ax[0].set_title('Basic normalization')
ax[0].hlines(1000,xmin=0,xmax=2047,color=colors[0])
nimg_dose = bb.normalize(img,ob,dc, dose_roi = [500,500,550,550], logarithm=False)
ax[1].imshow(nimg_dose,vmin=0,vmax=1.25,cmap='gray')
ax[1].set_title('Normalization with dose correction')
ax[1].hlines(1000,xmin=0,xmax=2047,color=colors[1])

ax[2].plot(filt.median(nimg[1000],np.ones(N)),label='Raw normalization')
ax[2].set_title('Profiles')

ax[2].plot(filt.median(nimg_dose[1000],np.ones(N)),label='Normalization with dose correction')
ax[2].legend();

You can barely see any difference between the normalized images. When you look at the line profiles you can however see that there actually is a difference in the transmission value of between the two images. The one without dose correction has about 1.02 in the open beam area, which is not physically correct. After the dose correction, you can see that the average intensit in air is about one, but the air profile is not perfectly straight. There is a slight increse close to the sample. This is a phenomenon we will handle in the next section.

# Improving the quality of the normalization

In [ ]:
mask,r,c,regions = bb.get_black_bodies(bbob, greythres = 2000, areas=[500,1000],R=10)
plt.imshow(mask, cmap=cmap)


In [ ]:
regions.head()

In [ ]:
sob=bb.compute_scatter_image(bbob-dc,r,c)
ss=bb.compute_scatter_image(bbs-dc,r,c)
fig,ax = plt.subplots(1,3,figsize=(12,6))
ax[0].imshow(sob, vmin=ss.min(),vmax=ss.max())
ax[1].imshow(ss)
ax[2].imshow(bbob-dc-sob)

In [ ]:
sob=bb.compute_scatter_image_from_df(regions,img.shape)
ss=bb.compute_scatter_image_from_df(regions,img.shape)
fig,ax = plt.subplots(1,3,figsize=(12,6))
ax[0].imshow(sob, vmin=ss.min(),vmax=ss.max())
ax[1].imshow(ss)
ax[2].imshow(bbob-dc-sob)

## Detect dots

In [ ]:
a=bbs-dc

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(12,5))

ax[0].imshow(a[350:410,290:350])
ax[0].set_title('Dot cropped from image')
template=bb.get_dot_template((bbs-dc),roi=[350,290,410,350])
ax[1].imshow(template)
ax[1].set_title('Template dot');

## Estimate OB scattering image

In [ ]:
obmask,r,c,bbdf = bb.get_blackbodies_by_templatematching(bbob-dc,template,0.5)

obest=bb.compute_scatter_image_from_df(bbdf,obmask.shape)

cmap = 'coolwarm'
fig,ax=plt.subplots(1,3,figsize=(12,5))
df=bb.check_scatter_image(bbob-dc,0,obmask,ax=ax[0],cmap='viridis')
ax[0].set_title('Intensity behind BBs')
a1=ax[1].imshow(obest,cmap='viridis')
ax[1].set_title('Scatter estimate')
fig.colorbar(a1,ax=ax[1],shrink=0.5)
df=bb.check_scatter_image(bbob-dc,obest,obmask,ax=ax[2],cmap=cmap,sym_cmap=True)
ax[2].set_title('Measured - Estimated');

## Estimate scattering image with sample

In [ ]:
smask,r,c,sdf   = bb.get_blackbodies_by_templatematching(bbs-dc,template,0.5)
sest=bb.compute_scatter_image_from_df(sdf,smask.shape)

cmap = 'coolwarm'
fig,ax=plt.subplots(1,3,figsize=(12,5))
df=bb.check_scatter_image(bbs-dc,0,smask,ax=ax[0],cmap='viridis')
ax[0].set_title('Intensity behind BBs')
a1=ax[1].imshow(sest,cmap='viridis')
ax[1].set_title('Scatter estimate')
fig.colorbar(a1,ax=ax[1],shrink=0.5)
df=bb.check_scatter_image(bbs-dc,sest,smask,ax=ax[2],cmap=cmap,sym_cmap=True)
ax[2].set_title('Measured - Estimated');

## Normalization with scattering correction

$$\frac{\tilde{I}_n}{\tilde{I}_{OB}}=\frac{I_n}{I_{OB}}\cdot\frac{D(I_{OB})}{D(I_{n})}=\frac{I_{n}^{*}-I_{DC}-I_{n,BB}^{S}\frac{D\left(I_{n}^{*}-I_{DC}\right)}{D\left(I_{n,BB}^{*}-I_{DC}-\left(1-\frac{1}{\tau_{BB}}\right)I_{n,BB}^{S}\right)\tau_{BB}}}{I_{OB}^{*}-I_{DC}-I_{BG,BB}^{S}\frac{D\left(I_{OB}^{*}-I_{DC}\right)}{D\left(I_{OB,BB}^{*}-I_{DC}-\left(1-\frac{1}{\tau_{BB}}\right)I_{BG,BB}^{S}\right)\tau_{BB}}}\cdot\frac{D\left(I_{OB}^{*}-I_{DC}-I_{BG,BB}^{S}\frac{D\left(I_{OB}^{*}-I_{DC}\right)}{D\left(I_{OB,BB}^{*}-I_{DC}-\left(1-\frac{1}{\tau_{BB}}\right)I_{BG,BB}^{S}\right)\tau_{BB}}\right)}{D\left(I_{n}^{*}-I_{DC}-I_{n,BB}^{S}\frac{D\left(I_{n}^{*}-I_{DC}\right)}{D\left(I_{n,BB}^{*}-I_{DC}-\left(1-\frac{1}{\tau_{BB}}\right)I_{n,BB}^{S}\right)\tau_{BB}}\right)}$$

In [ ]:
# Projection normalized with dose correction
nimg_dose = bb.normalize(img,ob,dc, dose_roi = [500,500,550,550], logarithm=False)

# Projection normalized with scattering correction
nimg_sc   = bb.normalization_with_BB(img,ob,dc,bbs,bbob,sest,obest,[500,500,550,550],0.97)

In [ ]:
# Visualization 
N=25
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

fig, ax = plt.subplots(1,3,figsize=(15,4))

ax[0].imshow(nimg_dose,vmin=0,vmax=1.25,cmap='gray')
ax[0].set_title('Normalization with dose correction')
ax[0].hlines(1000,xmin=0,xmax=2047,color=colors[0])

ax[1].imshow(nimg_sc,vmin=0,vmax=1.25,cmap='gray')
ax[1].set_title('Normalization with scattering correction')
ax[1].hlines(1000,xmin=0,xmax=2047,color=colors[1])

ax[2].plot(filt.median(nimg_dose[1000:1010].mean(axis=0), np.ones(N)),  label='Normalization with dose correction')
ax[2].plot(filt.median(nimg_sc[1000:1010].mean(axis=0),   np.ones(N)),  label='Normalization with scattering correction')

ax[2].set_title('Profiles')

ax[2].legend();